<a href="https://colab.research.google.com/github/LuisEduardoRB/EDP-II/blob/main/M%C3%A9todo_de_Jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">

# ✨ Método de Jacobi para el Problema de Laplace ✨

*Problema de valores de frontera en una malla con paso \(h=\tfrac{2}{3}\).*

</div>

---

**Condiciones de frontera:**

Sea \(u(x,y)\) definida en el rectángulo [0,2]x[0,2]
En el contorno se tienen los valores de:

$$
\begin{aligned}
u(0,y) &= 0, & \quad 0 < y < 2, \\[2mm]
u(2,y) &= y(2-y), & \quad 0 < y < 2, \\[2mm]
u(x,0) &= 0, & \quad 0 < x < 2, \\[2mm]
u(x,2) &=
\begin{cases}
x, & 0 < x < 1, \\[2mm]
2 - x, & 1 \le x < 2.
\end{cases}
\end{aligned}
$$

**Tamaño de malla:** h=2/3


In [ ]:
# Jacobi para Laplace en [0,2]x[0,2] con h=2/3

def Malla(h):
    # nodos de la malla (en x y en y son iguales porque es cuadrada)
    xs=[0,h,2*h,3*h]
    ys=[0,h,2*h,3*h]
    return xs,ys

def Frontera(u,xs,ys):
    # mete las condiciones de frontera en la malla u
    n=len(xs)
    # lado izquierdo y abajo son 0
    for j in range(n):
        u[0][j]=0
    for i in range(n):
        u[i][0]=0
    # lado derecho: u(2,y)=y(2-y)
    for j in range(n):
        y=ys[j]
        u[-1][j]=y*(2-y)
    # lado de arriba: depende de x
    for i in range(n):
        x=xs[i]
        if 0<x<1:
            u[i][-1]=x
        else:
            u[i][-1]=2-x

def JL(h,tol,max):
    xs,ys=Malla(h)
    n=len(xs)
    # inicializo la malla con ceros
    u=[[0 for _ in range(n)] for _ in range(n)]
    # pongo las condiciones de frontera
    Frontera(u,xs,ys)
    # copio la malla para ir guardando lo nuevo
    nuevo=[[u[i][j] for j in range(n)] for i in range(n)]

    # encabezado para ver los valores en forma de tabla
    print(f"{'Iter':>5} {'Error':>12} {'u11':>12} {'u21':>12} {'u12':>12} {'u22':>12}")
    print("-"*75)

    for k in range(1,max+1):
        # aquí aplico la fórmula de Jacobi a los puntos interiores
        for i in range(1,n-1):
            for j in range(1,n-1):
                nuevo[i][j]=0.25*(u[i+1][j]+u[i-1][j]+u[i][j+1]+u[i][j-1])
        # calculo el error como el máximo cambio
        err=0
        for i in range(1,n-1):
            for j in range(1,n-1):
                dif=abs(nuevo[i][j]-u[i][j])
                if dif>err:
                    err=dif
        # actualizo los valores de u
        for i in range(1,n-1):
            for j in range(1,n-1):
                u[i][j]=nuevo[i][j]

        # imprimo una fila de la tabla con la iteración, error y valores interiores
        print(f"{k:5d} {err:12.6f} {u[1][1]:12.6f} {u[2][1]:12.6f} {u[1][2]:12.6f} {u[2][2]:12.6f}")

        # paro si el error ya es menor que la tolerancia
        if err<tol:
            break

    # al final muestro los nodos y la malla completa
    print("\nNodos x:",[round(x,6) for x in xs])
    print("Nodos y:",[round(y,6) for y in ys])
    print("\nMalla u(i,j):")
    for j in range(n-1,-1,-1):
        fila=[round(u[i][j],6) for i in range(n)]
        print("y=",round(ys[j],2),"|",fila)
    return u,xs,ys,k,err

# aplicamos las fórmulas que ya creamos previamente y le metemos los datos
u,xs,ys,iters,err=JL(h=2/3,tol=1e-6,max=2000)
print("\nTerminó en",iters,"iteraciones con error",err)
print("\nValores interiores finales:")
# muestro los 4 valores que eran incógnitas
interior=[(1,1),(2,1),(1,2),(2,2)]
for (i,j) in interior:
    print("u(",xs[i],",",ys[j],")=",round(u[i][j],6))


 Iter        Error          u11          u21          u12          u22
---------------------------------------------------------------------------
    1     0.388889     0.000000     0.222222     0.166667     0.388889
    2     0.097222     0.097222     0.319444     0.263889     0.486111
    3     0.048611     0.145833     0.368056     0.312500     0.534722
    4     0.024306     0.170139     0.392361     0.336806     0.559028
    5     0.012153     0.182292     0.404514     0.348958     0.571181
    6     0.006076     0.188368     0.410590     0.355035     0.577257
    7     0.003038     0.191406     0.413628     0.358073     0.580295
    8     0.001519     0.192925     0.415148     0.359592     0.581814
    9     0.000760     0.193685     0.415907     0.360352     0.582574
   10     0.000380     0.194065     0.416287     0.360731     0.582954
   11     0.000190     0.194255     0.416477     0.360921     0.583143
   12     0.000095     0.194350     0.416572     0.361016     0.583238
 